In [1]:
ENV["PYTHONPATH"] = "/home/gridsan/aligho/.local/lib/python3.8/site-packages/PyNormaliz-2.15-py3.8-linux-x86_64.egg";
D = 2;
using HDF5, PyPlot
using DelimitedFiles, PyPlot, LinearAlgebra, JLD2, PyCall, StaticArrays, Statistics
using Crystalline, Brillouin, MPBUtils, SymmetryBases
using Crystalline: TEST_αβγs, TEST_αβγ, dot, norm
using Crystalline: normscale, matching_littlegroups
using Crystalline: AbstractFourierLattice
topology_paper_dir = "../TopologyPaper/"
include(topology_paper_dir * "get-freqs-symeigs.jl")
include(topology_paper_dir * "symeigs-from-io.jl");
include("../Topology_ML/Phc_ML/fft.jl")
include("get_uc_coefs.jl");

In [2]:
data_dir = "../TopologyPaper/symeigs/"
filename = "sg2-data-g0.h5";

In [3]:
loaded_data = Dict{String, Any}[]
for epsid in 1:5
    dir = data_dir * "./output/sg2/eps$epsid/tm/";
    filename_jld = "sg2-epsid$(epsid)-res64-tm.jld2"
    loaded_data_per_gidx = load(dir * filename_jld)
    push!(loaded_data, loaded_data_per_gidx)
end

In [6]:
summariesv = vcat([x["summariesv"] for x in loaded_data]...);
first_band_summaries = [summaries[1] for summaries in summariesv];
second_band_summaries = [summaries[2] for summaries in summariesv];
third_band_summaries = [summaries[3] for summaries in summariesv];
first_band_topologies = [summary.topology for summary in first_band_summaries];
band_topology_count = [count(x-> x.topology == NONTRIVIAL, [summaries[band_idx] for summaries in summariesv]) for band_idx in 1:3] ./ 400

3-element Vector{Float64}:
 35.745
 45.67
 58.57

In [30]:
band_summaries = first_band_summaries
#band_summaries = second_band_summaries
#band_summaries = third_band_summaries

y_plus_idxs = findall(x->x[1] == 1 && x[end] == 1, [x.n for x in band_summaries]);
y_minus_idxs = findall(x->x[2] == 1 && x[end] == 1, [x.n for x in band_summaries]);

b_plus_idxs = findall(x->x[3] == 1 && x[end] == 1, [x.n for x in band_summaries]);
b_minus_idxs = findall(x->x[4] == 1 && x[end] == 1, [x.n for x in band_summaries]);

a_plus_idxs = findall(x->x[5] == 1 && x[end] == 1, [x.n for x in band_summaries]);
a_minus_idxs = findall(x->x[6] == 1 && x[end] == 1, [x.n for x in band_summaries]);

gamma_plus_idxs = findall(x->x[7] == 1 && x[end] == 1, [x.n for x in band_summaries]);
gamma_minus_idxs = findall(x->x[8] == 1 && x[end] == 1 , [x.n for x in band_summaries]);

In [4]:
f = h5open(topology_paper_dir  * "sg2-data-g0.h5", "r");

In [5]:
epsilon_Gs = vcat([[f["sg2/$(i)/epsilon_Gs-epsid=$(epsid)"][] for i in 1:10000] for epsid in 1:5]...);
epsilon_Gs_signs = [real.(x ./abs.(x)) for x in epsilon_Gs];

In [32]:
round.(100 * [count(x -> x[i] >  0,  epsilon_Gs_signs[b_plus_idxs]) + count(x -> x[i] <  0,  
            epsilon_Gs_signs[b_minus_idxs]) for i in 2:10] / (length(b_plus_idxs)+length(b_minus_idxs)), digits=0)

9-element Vector{Float64}:
 50.0
 88.0
 50.0
 50.0
 51.0
 50.0
 50.0
 51.0
 50.0

In [33]:
round.(100 * [count(x -> x[i] >  0,  epsilon_Gs_signs[y_plus_idxs]) + count(x -> x[i] <  0,  
            epsilon_Gs_signs[y_minus_idxs]) for i in 2:10] / (length(y_plus_idxs)+length(y_minus_idxs)), digits=0)

9-element Vector{Float64}:
 90.0
 50.0
 50.0
 51.0
 50.0
 50.0
 50.0
 49.0
 50.0

# Delineating by contrast 

In [54]:
epsid = 5
band_summaries = first_band_summaries[(1+(epsid-1)*10000):((epsid*10000))]

y_plus_idxs = findall(x->x[1] == 1 && x[end] == 1, [x.n for x in band_summaries]);
y_minus_idxs = findall(x->x[2] == 1 && x[end] == 1, [x.n for x in band_summaries]);

b_plus_idxs = findall(x->x[3] == 1 && x[end] == 1, [x.n for x in band_summaries]);
b_minus_idxs = findall(x->x[4] == 1 && x[end] == 1, [x.n for x in band_summaries]);

a_plus_idxs = findall(x->x[5] == 1 && x[end] == 1, [x.n for x in band_summaries]);
a_minus_idxs = findall(x->x[6] == 1 && x[end] == 1, [x.n for x in band_summaries]);

gamma_plus_idxs = findall(x->x[7] == 1 && x[end] == 1, [x.n for x in band_summaries]);
gamma_minus_idxs = findall(x->x[8] == 1 && x[end] == 1 , [x.n for x in band_summaries]);

In [55]:
round.(100 * [count(x -> x[i] >  0,  epsilon_Gs_signs[(1+(epsid-1)*10000):((epsid*10000))][b_plus_idxs]) + count(x -> x[i] <  0,  
            epsilon_Gs_signs[(1+(epsid-1)*10000):((epsid*10000))][b_minus_idxs]) for i in 2:10] / (length(b_plus_idxs)+length(b_minus_idxs)), digits=0)

9-element Vector{Float64}:
 50.0
 88.0
 50.0
 50.0
 51.0
 50.0
 50.0
 51.0
 50.0